# CS182 HW3 Coding [40 points]

In this coding homework, you will be required to complete several models for binary classification and try to find the inplicit relationship of them by yourself. 

**Good luck!**


In [10]:
from sklearn import svm
from scipy import special
import numpy as np

In [ ]:
X_train = np.loadtxt('data/X_train.txt')
X_val = np.loadtxt('data/X_val.txt')
X_test = np.loadtxt('data/X_test.txt')
y_train = np.loadtxt('data/y_train.txt')
y_val = np.loadtxt('data/y_val.txt')
y_test = np.loadtxt('data/y_test.txt')

w = np.loadtxt('data/w.txt')
w0 = np.loadtxt('data/w0.txt')

## (a) Simple Perceptron

(1) Activation functions and loss functions are important parts of each neural network, and there are multiple ways of calculating them. 

 **[3 points]** In this question, we ask you to implement the **sigmoid function** and **binary cross entroy loss function** serving for the binary classification.

In [4]:
def sigmoid(x):


def BCEloss(y_pred, y):


(2) **[3 points]** In this question, we ask you to implement the **softmax function** and **cross entroy loss function** serving for the multiple classification.

In [5]:
def softmax(x):


def cross_entropy_loss(y,y_pre):


(3) **[10 points]** Learning a simple perceptron with **batch GD** (using the given initializations $w^{init}$ and $w^{init}_{0}$) based on the training set ($X_{train}$, $y_{train}$): use the training set and the validation set to obtain a good learning rate (you can set the maximum for iterations to 50 and try different learning rate in [$10^{−4}$,  $10^{−8}$] ); output the learned model and evaluate its performance on the test set with the classification accuracy.

In [ ]:
# BGD Implementation


In [ ]:
# Evaluation with Sigmoid function


(4) **[10 points]** Learning a simple perceptron with **SGD** (using the given initializations $w^{init}$ and $w^{init}_{0}$) based on the training set ($X_{train}$, $y_{train}$): use the training set and the validation set to obtain a good learning rate(you can set the maximum for iterations and try different learning rate); output the learned model and evaluate its performance on the test set with the classification accuracy.

In [ ]:
# SGD Implementation


In [ ]:
# Evaluation with Sigmoid function


## (b) SVM

(1) **[10 points]** Use the function **‘svm’** in package **‘sklearn’** to do the binary classification. Output the model and evaluate its performance on each dataset with the classification accuracy.

In [ ]:
# SVM Implementation


## (c) Compare

(1) **[4 points]** Try to compare  models learned from (a)(3), (a)(4) and (b). Write down your explanation and data support.